## Imports

In [50]:
import pandas as pd
import numpy as np
from utils import *;
from queryAnswer import *;
from IPython.display import *;
import re
import json
import os

## Helpers

In [138]:
def readDict(file):
    with open(f"dictionaries/{file}.json") as infile:
        subs = json.load(infile)
    subs = {int(k) : v for k, v in subs.items()}
    return subs
def writeDict(dct, file):
    with open(f"dictionaries/{file}.json", "w") as outfile:
        json.dump(dct, outfile, indent=4, sort_keys=True)
def addDict(file=None, dct = None, keys = None, values = None):
    if not dct: 
        dct = readDict(file)
    if keys and values:
        if type(keys) != list:
            dct[keys] += values
        else:
            assert len(keys) == len(values), \
            f"""Keys and values must have same length:
                Given Keys:{len(keys)} and Values:{len(values)}"""
            for key, value in zip(keys, values):
                dct[key] += value  
    writeDict(dct, file)

## Variables

### link

In [139]:
base_url = "https://erowid.org/experiences/research/"
api_code = "exp_api.php?api_code=berkeley_bcsp_tyrone_2022"
link = base_url + api_code

In [ ]:
def retrieve(subID, most=20, conditions = None):
    retrieved = readDict("retrieved")
    retrieved[subID] += []
    retrieved[subID] += lst
    

### dictionaries

In [184]:
substance_key = readDict("substances")
retrieved = readDict("retrieved")

In [185]:
exp_list(1, most = 1000, conditions = ["exp.isFirst()", "exp.isPure()"]);

In [183]:
def exp_list(subID, most=20, conditions = None):
    dct = getExp(subID, most=most, conditions = conditions)
    return list(dct.values())

In [181]:
def getExp(subID, most=20, conditions=None):
    add =  "&a=experience_data" + \
          f"&substance_id={subID}&max={most}" + \
           "&format=xml"
    url = link + add 
    val_dict = {}
    content = requests.get(url).text
    exps = re.split(r"<experience>|</experience>", content)[:-1]
    info, exps = exps[0], exps[1:]
    ids = re.findall("<substance-id-list>(.*)</substance-id-list>", info)[0]
    ids = list(eval(ids))
    experiences = readDict("experiences")
    while ("\n\n\n" in exps):
        exps.remove("\n\n\n")
    dct = {}
    for i in range(len(exps)):
        exp = Experience(exps[i])
        experiences[exp.get("id")] = vars(exp)
        fits = True
        for condition in conditions:
            met = eval(condition)
            fits = fits and met
        if fits:
            dct[exp.name] = exp
    writeDict(experiences, "experiences")
    return dct

## Pre-Processing

In [182]:
class Experience:
    
    def fromDict(dct : dict):
        return Experience(**dct)
    
    def __repr__(self):
        dct  = self.info
        ss, ln = dct["substance-string"].strip(), dct["list-number"]
        return f"Experience: {{ Substance: {ss},  List-ID: {ln} }}"
        
    def __str__(self):
        return self.name
        
    def __init__(self, string : str, 
                 info = None, text = None, name = None):
        self.string = string
        self.readInfo()
        for i, k in self.info.items():
            if self.info.get(i) == "":
                self.info[i] = None
        self.makeName()
    
    def get(self, key : str):
        if key in (inf := self.info):
            return inf.get(key)
        else:
            print(f"{key} is not a valid key.",
                  "Please select one of the following: \n")
            for key in inf.keys():
                if key == "text":
                    print(key)
                else:
                    print(key, end = "," + " "*(30 - len(key)) + "\t")
                    
    def isPure(self):
        return type(self.get("substance-id-list")) == int
    
    def isFirst(self):
        return 2 in self.get("category-id-list")
        
    def readInfo(self):
        vallst = re.findall(r"<(.*)>(.*)</.*>", self.string)
        dct = dict(vallst)
        for key in dct.keys():
            try:
                dct[key] = eval(dct[key])
            except Exception as e:
                dct[key] = dct[key]
        self.info = dct
        txt = re.split(r"<experience-text>|</experience-text>", self.string)[1]
        self.info['text'] = txt
        self.text = txt
        self.info["list-number"] = self.info["list-number"] - 1
        
    def makeName(self):
        dct = self.info
        name = dct["substance-string"].strip() + str(dct["list-number"])
        self.name = name.lower().replace(" ", "_").replace("&", "n")
        
    def listInfo(self):
        return list(self.info.keys())

    def toJSON(self):
        return json.dumps(self, default=lambda o: vars(o), 
            sort_keys=True, indent=4)

https://erowid.org/experiences/research/exp_api.php?api_code=berkeley_bcsp_tyrone_2022&substance_id=1&a=experience_data&max=1